In [ ]:
from matplotlib import pyplot as pyplot
import numpy as np
import seaborn as sns

In [51]:
WIDTH = 10
HEIGHT = 5
board = np.random.choice([0,1], WIDTH * HEIGHT).reshape((HEIGHT,WIDTH))
board

array([[0, 1, 1, 0, 1, 1, 1, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 1, 0, 0, 1],
       [1, 0, 0, 1, 1, 0, 1, 1, 0, 0],
       [0, 0, 1, 0, 1, 1, 0, 0, 1, 0],
       [1, 0, 0, 1, 0, 0, 1, 1, 0, 0]])

In [53]:
for row_ix, row in enumerate(board):
    for col_ix, cell in enumerate(row):
        left_col = max(0, col_ix - 1)
        right_col = min(col_ix + 2, WIDTH)
        top_row = max(0, row_ix - 1)
        bottom_row = min(row_ix + 2, HEIGHT)
        neighborhood = board[top_row:bottom_row, left_col:right_col]
        cell_col_loc = 1 if col_ix > 0 else 0
        cell_row_loc = 1 if row_ix > 0 else 0
        cell_loc = cell_row_loc, cell_col_loc

        

   

In [ ]:
board

In [ ]:
neighborhood